# Efficient Exploration through ALPaCA Q Learning
Course project for CS332: Advanced Survey on Reinforcement Learning

Fengjun Yang, 2018

In [3]:
%reload_ext autoreload
%autoreload 2

# ALPaCA
from agent.alpaca import ALPaCA
from agent.alpacaQ import ALPaCAQ
from agent.alpacaoffline import ALPaCAOffline

# Mountain Car related
from metamountaincar.mmcenv import MetaMountainCar as MMC
from metamountaincar.utility import *
from metamountaincar.VI import mmcVI
from metamountaincar.mmcDataset import mmcDataset, MCOfflineDataset

# Utility
import numpy as np
import yaml
from tqdm import tqdm

In [ ]:
# Load configuration file
cfg_filename = 'config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

## Generate value functions for offline training. 
Technically, one can use mmcDataset directly for ALPaCA offline training. Here, however, we store and reuse sampled data.

SKIP IF USING PRESAMPLED DATA

In [ ]:
N_DATASET = 5                       # number of datasets to presample
N_FUNCS = config['meta_batch_size'] # number of environments per dataset
N_SAMPLES = 80                      # number of data points per environment
DATA_DIR = config['data_dir']       # where to store the presampled datasets (has to be an existing directory)

In [ ]:
# Sample Q functions and store in DATA_DIR
for i in tqdm(range(N_DATASET)):
    sample = mmcDataset(config).sample(N_FUNCS, N_SAMPLES)
    with open(DATA_DIR+str(i)+'.yml', 'w') as yml_file:
        yml_file.write(yaml.dump(sample))

## Offline training

In [ ]:
# Load dataset for offline training
dataset = MCOfflineDataset(config)

In [ ]:
# Initialize offline alpaca agent
offline = ALPaCAOffline(config)

In [ ]:
# Offline training
offline.train_offline(dataset, 1000)

In [ ]:
# Save agent in a specified directory
offline.save_agent('./saveagent/')

## Testing plotting functions

In [10]:
env = MMC()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [26]:
plot_true_cost_to_go(env, 115, 200)

(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)
(0, 57)


IndexError: index 115 is out of bounds for axis 1 with size 115

## Online prediction with analytical solution

In [ ]:
# Extract information from agent
K0 = sess1.run(agent.K)
L0 = sess1.run(agent.L)
L0_inv = np.linalg.inv(L0)
Sig = sess1.run(agent.SigEps)
encode = agent.encode

In [ ]:
# Predict a specific point
state = np.array([0.5, 0.06])
norm_ob = dataset.normalize_x(state[None,None,:])
phi = encode(norm_ob)[0].T
pred_mean = dataset.denormalize_y(K0.T @ phi)
pred_var = (1 + phi.T @ L0_inv @ phi) * np.diag(Sig[0,0]) * (dataset.q_var ** 2)
pred_mean.flatten(), pred_var

In [ ]:
# Update the model with some test online data
# Suppose online data is: (for action 0)
# - (0.5,0.06) --> 1
action = 0
X = np.array([[0.5,0.06]]).T
Y = np.array([[0]])
# Preprocess the data
norm_X = dataset.normalize_x(X.T[None,:,:])
Phi = agent.encode(norm_X)[0].astype(np.float64)
norm_Y = dataset.normalize_y(Y[None,:,:])

In [ ]:
# Predict the same point after the observations using the analytical solution
L_ana = (Phi.T @ Phi + L0).astype(np.float64)
L_ana_inv = np.linalg.inv(L_ana)
K_ana = K0.copy().astype(np.float64)
Q_ana = Phi.T @ norm_Y + (L0 @ (K0[:, action]))[:, None]
K_ana[:, action] = (L_ana_inv @ Q_ana)[0,:,0]
pred_mean = dataset.denormalize_y(K_ana.T @ phi)
pred_var = (1 + phi.T @ L_ana_inv @ phi) * np.diag(Sig[0,0]) * (dataset.q_var ** 2)
pred_mean.flatten(), pred_var

## Online prediction with incremental update

In [ ]:
# Predict the same point after incremental update
aq = ALPaCA_Q(agent, None, config, sess1)

In [ ]:
aq.update_model(X.T[0],action,norm_Y[0])
target = dataset.normalize_y(0)
aq.predict_q_values(state).flatten()

In [ ]:
aq.predict_var(state) * dataset.q_var ** 2

## Plot true q-table

In [ ]:
# Sample or specify a gravity - thrust pair
gravity, thrust = 0.0025, 0.001

In [ ]:
# Compute VI for this configuration
gran = 150
VIagent = MCVI(gran, 200, gravity=grav, thrust=thrust)
VIagent.value_iteration()
Q_table = VIagent.q_table

In [ ]:
# Plot
plot_true_cost_to_go(Q_table)

## Plot learned q function

In [ ]:
def plot_learned_cost_to_go(ALPaCA_Q, grid, savedir=None):
    """
    Plot the learned cost to go function for an ALPaCA_Q agent
    """
    # Process the data for plotting
    X = np.arange(-1.2, 0.6, 1.8/grid)
    Y = np.arange(-0.07, 0.07, 0.14/grid)
    X,Y = np.meshgrid(X,Y)
    # Fill in the table by iteratively querying the agent
    Z = np.zeros(X.shape)
    for i in range(grid):
        for j in range(grid):
            Z[i,j] = ALPaCA_Q.predict_q_values(np.array([X[i,j],Y[i,j]])).max()
    # Plotting
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X,Y,-Z, cmap=cm.coolwarm)
    if savedir is not None:
        fig.savefig(savedir,transparent=True)

In [ ]:
def plot_learned_variance(ALPaCA_Q, grid, savedir=None):
    """
    Plot the learned cost to go function for an ALPaCA_Q agent
    """
    # Process the data for plotting
    X = np.arange(-1.2, 0.6, 1.8/grid)
    Y = np.arange(-0.07, 0.07, 0.14/grid)
    X,Y = np.meshgrid(X,Y)
    # Fill in the table by iteratively querying the agent
    Z = np.zeros(X.shape)
    for i in range(grid):
        for j in range(grid):
            s = np.array([X[i,j],Y[i,j]])
            Z[i,j] = ALPaCA_Q.predict_var(s).mean()
    # Plotting
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X,Y,np.sqrt(Z), cmap=cm.coolwarm)
    if savedir is not None:
        fig.savefig(savedir,transparent=True)

In [ ]:
# Define an agent
agent.restore('./Pretrained_alpaca/norm_y_2/')
aq = ALPaCA_Q(agent, None, config, sess1)

In [ ]:
plot_learned_cost_to_go(MCaq, 20)

In [ ]:
plot_learned_variance(aq,20)

## Mountain Car Experiment / RL

In [ ]:
def plot_summary(result):
    avg_result = result.mean(0)
    ind = np.arange(len(avg_result)) + 1
    plt.plot(ind, avg_result)
    plt.show()

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Create environment and ALPaCA Q agent
env = MCF()
agent.restore('./Pretrained_alpaca/norm_y_2/')
aq = ALPaCA_Q(agent, None, config, sess1)
MCaq = ALPaCA_Q(agent, env, config, sess1)

In [ ]:
grav, thru = (0.0021448523969069813, 0.0012198799893080752)
env1 = MCF(gravity=grav, thrust=thru)
grav, thru = (0.0023464856819864063, 0.0013694856651231633)
env2 = MCF(gravity=grav, thrust=thru)

In [ ]:
num_episodes = 100
step_limit = 500
for i in range(num_episodes):
    MCaq.ts_non_bootstrap(step_limit)
    print(play_episode(env, MCaq, render=False,it=step_limit))

In [ ]:
%matplotlib notebook

In [ ]:
plot_learned_cost_to_go(MCaq, 20)

In [ ]:
# E-greedy learning env1
num_experiments = 20
num_episodes = 10
num_plays = 3
step_limit = 200
env = env1
e_greedy_results_env1 = np.zeros((num_experiments, num_episodes))
# Run experiment
for i in tqdm(range(num_experiments)):
    MCaq = ALPaCA_Q(agent, env, config, sess1) #Reinitialize agent for each experiment
    for j in range(num_episodes):
        train_step = MCaq.e_greedy_train_episode(step_limit,render=False)
        # Play several games to find average steps
        play_step = []
        for k in range(num_plays):
            play_step.append(play_episode(env, MCaq, render=False,it=step_limit))
        e_greedy_results_env1[i][j] = np.mean(play_step)

In [ ]:
# E-greedy learning env2
num_experiments = 20
num_episodes = 1000
num_plays = 3
step_limit = 200
env = env2
e_greedy_results_env2 = np.zeros((num_experiments, num_episodes))
# Run experiment
for i in tqdm(range(num_experiments)):
    MCaq = ALPaCA_Q(agent, env, config, sess1) #Reinitialize agent for each experiment
    for j in range(num_episodes):
        train_step = MCaq.e_greedy_train_episode(step_limit,render=False)
        # Play several games to find average steps
        play_step = []
        for k in range(num_plays):
            play_step.append(play_episode(env, MCaq, render=False,it=step_limit))
        e_greedy_results_env2[i][j] = np.mean(play_step)

In [ ]:
# TS learning env1
num_experiments = 10
num_episodes = 1000
num_plays = 3
step_limit = 200
env = env1
# Array to store results
TS_results_env1 = np.zeros((num_experiments, num_episodes))
# Run experiment
for i in tqdm(range(num_experiments)):
    MCaq = ALPaCA_Q(agent, env, config, sess1)
    for j in range(num_episodes):
        train_step = MCaq.ts_train_episode(step_limit,render=False)# Train an episode and get data
        # Play several games to find average steps
        play_step = []
        for k in range(num_plays):
            play_step.append(play_episode(env, MCaq, render=False,it=step_limit))
        TS_results_env1[i][j] = np.mean(play_step)

In [ ]:
plot_summary(TS_results_env1)

In [ ]:
# TS learning env2
num_experiments = 10
num_episodes = 1000
num_plays = 3
step_limit = 200
env = env2
# Array to store results
TS_results_env2 = np.zeros((num_experiments, num_episodes))
# Run experiment
for i in tqdm(range(num_experiments)):
    MCaq = ALPaCA_Q(agent, env, config, sess1)
    for j in range(num_episodes):
        train_step = MCaq.ts_train_episode(step_limit,render=False)# Train an episode and get data
        # Play several games to find average steps
        play_step = []
        for k in range(num_plays):
            play_step.append(play_episode(env, MCaq, render=False,it=step_limit))
        TS_results_env2[i][j] = np.mean(play_step)

In [ ]:
eg_1 = np.loadtxt('Plots/egreedyresult1backup.txt').mean(0)
ts_1 = TS_results_env1.mean(0)
ind = np.arange(1000) + 1
l1, = plt.plot(ind, eg_1)
l2, = plt.plot(ind, ts_1)
l3, = plt.plot(ind, [90]*1000, '-')
plt.legend([l1, l2, l3], ['$\epsilon$-greedy', 'Thompson sampling', 'Optimal policy'])
plt.xlabel('Episodes')
plt.ylabel('Average steps per episode')
plt.savefig('Plots/RL_result1.png', dpi=500)

In [ ]:
MCaq.L_inv

In [ ]:
np.savetxt('Plots/TS-results-backup.txt',TS_results_env1)

In [ ]:
plot_summary(e_greedy_results_backup_2)

In [ ]:
plot_learned_cost_to_go(MCaq, grid=25)

In [ ]:
plot_learned_variance(MCaq,grid=25)

In [ ]:
e_greedy_summary = e_greedy_results_env2[:18].mean(0)
plt.plot(np.arange(num_episodes)+1, e_greedy_summary)
#plt.savefig('Plots/e-greedy-steps0.png')

In [ ]:
#env1_params_backup
#env2_params_backup
#e_greedy_results_backup_1 #500 episode original env
#e_greedy_results_backup_2 #1000 episode env1, 20 experiments, epsilon=0.3
#e_greedy_results_backup_3 #1000 episode env2, 19 experiment, epsilon=0.3

## Clean up

In [ ]:
env.close()
env1.close()
env2.close()

In [ ]:
sess1.close()